In [4]:
import re
def clean_text(text):
    
    text = re.sub(r'<.*?>', '', text)  # remove HTML tags
    text = re.sub(r'[^\w\s]', '', text)  # remove punctuation
    text = text.lower()  # convert to lowercase
    return text

def preprocess_chunk(chunk):
    # Select relevant columns
    chunk = chunk[['text', 'stars']]

    # Clean text data
    chunk['text'] = chunk['text'].apply(clean_text)

    # Convert stars to binary sentiment labels (1-3 stars as 0, 4-5 stars as 1)
    chunk['sentiment'] = chunk['stars'].apply(lambda x: 1 if x > 3 else 0)

    # Handle missing values
    chunk.dropna(inplace=True)

    return chunk

In [ ]:
import pandas as pd

#load the dataset
#[to run in google colab]

from google.colab import drive
drive.mount('/content/drive')

reviews_path = '/content/drive/My Drive/cw2-dataset/yelp_academic_dataset_review.json'

chunk_size = 100000
chunks = pd.read_json(reviews_path, lines=True, chunksize=chunk_size)


In [ ]:
#tokenisation

from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def tokenize_chunk(chunk):
    tokenized_input = tokenizer(
        chunk['text'].tolist(),
        add_special_tokens=True,
        max_length=512,
        truncation=True,
        padding='max_length',
        return_attention_mask=True,
        return_token_type_ids=False,
        return_tensors='pt'
    )
    return tokenized_input
